# Основные функции для наших моделей
Комплект функций уменьшающий вес набора данных! Нужно при больших данных

https://www.youtube.com/watch?v=5Al4pXdJmAY&ab_channel=DataStartConference

In [1]:

def mem_usage(obj):
    """    
    input: pd.DataFrame | pd.Series
    
    return: sting
    """
    #если DataFrame
    if isinstance(obj, pd.DataFrame):
        usage_b = obj.memory_usage(deep = True).sum()
    #если Series
    else:
        usage_b = obj.memory_usage(deep = True)
    usage_mb = usage_b / 1024 ** 2 # в МБ
    return "{:03.2f} MB".format(usage_mb)


def replace_cat(obj, float_cols, min_value=0.001, features=None):
    """
    Объекты в категории
    
    input: pd.DataFrame | pd.Series
    input: hasattr(features, '__iter__')
    input: .0 =< min_value >= 1.
    
    return pd.DataFrame | pd.Series
    """
    #только с DataFrame
    if isinstance(obj, pd.DataFrame):
        converted_obj = pd.DataFrame()
        if not features: features = obj.columns
        #обзор колонок
        for col in features:
            if col not in float_cols:
                num_unique_values = len(obj[col].unique())
                num_total_values = len(obj[col])
                #если % меньше
                if num_unique_values / num_total_values < min_value:
                    converted_obj.loc[:,col] = obj[col].astype('category')
                else:
                    converted_obj.loc[:,col] = obj[col]
            else:
                converted_obj.loc[:,col] = obj[col]
                
        return converted_obj
    #если Series
    else:
        if col not in float_cols:
            num_unique_values = len(obj[col].unique())
            num_total_values = len(obj[col])
            #если % меньше
            if num_unique_values / num_total_values < min_value:
                return obj[col].astype('category')
            else:
                return obj[col]
        
        
def as_dict(frame):
    """
    перевод в словарь
    
    input: pd.DataFrame
    
    return dictionary
    """
    #процесс образования словаря
    dtypes = frame.dtypes
    dtypes_col = dtypes.index
    dtypes_type = [i.name for i in dtypes.values]
    column_types = dict(zip(dtypes_col, dtypes_type))
    return {key:value for key,value in list(column_types.items())}


def weight_reducer(frame, min_value=0.001, get_dict = None):
    """
    Основная функция редьюсера
    
    input: pd.DataFrame
    input: .0 =< min_value >= 1.
    input: get_dict == bool
    
    return pd.DataFrame  
    """
    #если это Frame
    if isinstance(frame, pd.DataFrame):
        before = mem_usage(frame)
        float_cols = []
        for dtype in set(frame.dtypes):
            if 'float' in str(dtype):
                cols = frame[[col for col in frame.columns if 'float' in str(frame[col].dtypes)]].select_dtypes(include = [dtype]).columns
                frame[cols] = frame[cols].select_dtypes(include = [dtype]). \
                              apply(pd.to_numeric, downcast='float')
                float_cols = cols
            if 'int' in str(dtype): 
                cols = frame[[col for col in frame.columns if 'int' in str(frame[col].dtypes)]].select_dtypes(include = [dtype]).columns
                frame[cols] = frame[cols].select_dtypes(include = [dtype]).\
                              apply(pd.to_numeric, downcast='unsigned')
            frame = replace_cat(frame, float_cols, min_value)
        after = mem_usage(frame)
        print('Columns - {}, early size - {}, replaced size - {}'.format(frame.shape[1], before, after))
        if get_dict: frame = as_dict(frame)
        return frame